In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from datetime import datetime
import os
from utils import train_model, eval_model, SmoothedDataset, certificate_over_dataset
from attack_lib import attack_setting
from hashlib import sha256


In [2]:
args = {
    'dataset': 'cifar',
    'pair_id': 0,
    'atk_method': 'onepixel',
    'poison_r': 0.1,
    'delta': 1,
    'N_m': 10,
    'sigma': 1,
}
PREFIX = './saved_model/%s%s-%s(%.4f)-pr%.4f-sigma%.4f'%(args['dataset'], 'binary', args['atk_method'], args['delta'], args['poison_r'], args['sigma'])

np.random.seed(42)
torch.manual_seed(42)
poisoned_train, testloader_benign, testloader_poison, BATCH_SIZE, N_EPOCH, LR, Model = attack_setting(args, test_label_poison=True)


Non-target testset size: 1000
testset size: 2000
Poisoned testset size: 1000


In [3]:
for _ in range(args['N_m']):
        model = Model(gpu=True)
        model.load_state_dict(torch.load(PREFIX+'/smoothed_%d.model'%_))
        if args['sigma'] != 0:
            hashval = int(sha256(open(PREFIX+'/smoothed_%d.model'%_, 'rb').read()).hexdigest(), 16) % (2**32)
            model.fix_pert(sigma=args['sigma'], hash_num=hashval)
        acc_benign = eval_model(model, testloader_benign)
        acc_poison = eval_model(model, testloader_poison)
        if args['sigma'] != 0:
            model.unfix_pert()
        print ("Benign/Poison ACC %.4f/%.4f"%(acc_benign, acc_poison))


Benign/Poison ACC 0.6540/0.6480
Benign/Poison ACC 0.6985/0.5520
Benign/Poison ACC 0.6680/0.6080
Benign/Poison ACC 0.6665/0.5940
Benign/Poison ACC 0.6630/0.6320
Benign/Poison ACC 0.7700/0.2600
Benign/Poison ACC 0.7715/0.1490
Benign/Poison ACC 0.5110/0.1730
Benign/Poison ACC 0.7450/0.4280
Benign/Poison ACC 0.5435/0.9030


In [5]:
poisoned_train, testloader_benign, testloader_poison, BATCH_SIZE, N_EPOCH, LR, Model = attack_setting(args, test_label_poison=False)
pa_exp, pb_exp, is_acc = certificate_over_dataset(model, testloader_poison, PREFIX, args['N_m'], args['sigma'])
print(f'PA: {pa_exp.mean()* 100:.2f}%, PB: {pb_exp.mean()* 100:.2f}%, RA: {is_acc.mean()* 100:.2f}')
heof_factor = np.sqrt(np.log(1/0.0001)/2/args['N_m'])
pa = np.maximum(1e-8, pa_exp - heof_factor)
pb = np.minimum(1-1e-8, pb_exp + heof_factor)
from scipy.stats import norm

# Calculate the metrics
cert_bound = 0.5 * args['sigma'] * (norm.ppf(pa) - norm.ppf(pb))/np.sqrt(args['poison_r'])
cert_bound_exp = 0.5 * args['sigma'] * (norm.ppf(pa_exp) - norm.ppf(pb_exp))/np.sqrt(args['poison_r']) # Also calculate the bound using expected value.
rad = [args['delta']]
cert_acc = []
cond_acc = []
cert_ratio = []
cert_acc_exp = []
cond_acc_exp = []
cert_ratio_exp = []
for r in rad:
    cert_acc.append(np.logical_and(cert_bound>r, is_acc).mean())
    cond_acc.append(np.logical_and(cert_bound>r, is_acc).sum() / (cert_bound>r).sum())
    cert_ratio.append((cert_bound>r).mean())
    cert_acc_exp.append(np.logical_and(cert_bound_exp>r, is_acc).mean())
    cond_acc_exp.append(np.logical_and(cert_bound_exp>r, is_acc).sum() / (cert_bound_exp>r).sum())
    cert_ratio_exp.append((cert_bound_exp>r).mean())
print ("Certified Radius:", ' / '.join([str(r) for r in rad]))
print ("Cert acc:", ' / '.join(['%.5f'%x for x in cert_acc]))
print ("Cond acc:", ' / '.join(['%.5f'%x for x in cond_acc]))
print ("Cert ratio:", ' / '.join(['%.5f'%x for x in cert_ratio]))
print ("Expected Cert acc:", ' / '.join(['%.5f'%x for x in cert_acc_exp]))
print ("Expected Cond acc:", ' / '.join(['%.5f'%x for x in cond_acc_exp]))
print ("Expected Cert ratio:", ' / '.join(['%.5f'%x for x in cert_ratio_exp]))


Non-target testset size: 1000
testset size: 2000
Poisoned testset size: 1000


100%|██████████| 10/10 [00:02<00:00,  4.01it/s]

Attack Success Rate (ASR): 0.3950
PA: 66.10%, PB: 33.90%, RA: 60.50
Certified Radius: 1
Cert acc: 0.00000
Cond acc: nan
Cert ratio: 0.00000
Expected Cert acc: 0.38500
Expected Cond acc: 0.63742
Expected Cert ratio: 0.60400



C:\Users\phant\AppData\Local\Temp\ipykernel_42184\3265059726.py:21: RuntimeWarning: invalid value encountered in scalar divide
  cond_acc.append(np.logical_and(cert_bound>r, is_acc).sum() / (cert_bound>r).sum())
